In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('amazon.csv', encoding='utf-8')
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nColumn names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (1465, 16)

First few rows:
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,

In [2]:
# Step 1: Clean and convert price columns
# Remove ₹ symbol and commas, convert to numeric

def clean_price(price):
    """Remove currency symbol and commas, convert to numeric"""
    if pd.isna(price):
        return np.nan
    price = str(price).replace('₹', '').replace(',', '').strip()
    try:
        return float(price)
    except:
        return np.nan

df['discounted_price'] = df['discounted_price'].apply(clean_price)
df['actual_price'] = df['actual_price'].apply(clean_price)

print("Price columns cleaned:")
print(f"Discounted price - Min: {df['discounted_price'].min()}, Max: {df['discounted_price'].max()}")
print(f"Actual price - Min: {df['actual_price'].min()}, Max: {df['actual_price'].max()}")
print(f"Missing values - Discounted: {df['discounted_price'].isna().sum()}, Actual: {df['actual_price'].isna().sum()}")

Price columns cleaned:
Discounted price - Min: 39.0, Max: 77990.0
Actual price - Min: 39.0, Max: 139900.0
Missing values - Discounted: 0, Actual: 0


In [3]:
# Step 2: Clean discount_percentage
# Remove % symbol and convert to numeric

def clean_discount_percentage(discount):
    """Remove % symbol and convert to numeric"""
    if pd.isna(discount):
        return np.nan
    discount = str(discount).replace('%', '').strip()
    try:
        return float(discount)
    except:
        return np.nan

df['discount_percentage'] = df['discount_percentage'].apply(clean_discount_percentage)

print("Discount percentage cleaned:")
print(f"Min: {df['discount_percentage'].min()}, Max: {df['discount_percentage'].max()}")
print(f"Missing values: {df['discount_percentage'].isna().sum()}")

Discount percentage cleaned:
Min: 0.0, Max: 94.0
Missing values: 0


In [4]:
# Step 3: Clean rating_count
# Remove commas and convert to numeric

def clean_rating_count(count):
    """Remove commas and convert to numeric"""
    if pd.isna(count):
        return np.nan
    count = str(count).replace(',', '').strip()
    try:
        return int(count)
    except:
        return np.nan

df['rating_count'] = df['rating_count'].apply(clean_rating_count)

print("Rating count cleaned:")
print(f"Min: {df['rating_count'].min()}, Max: {df['rating_count'].max()}")
print(f"Missing values: {df['rating_count'].isna().sum()}")

Rating count cleaned:
Min: 2.0, Max: 426973.0
Missing values: 2


In [5]:
# Step 4: Ensure rating is numeric and handle missing values
# Convert rating to numeric and handle nulls

df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

print("Rating column analysis:")
print(f"Data type: {df['rating'].dtype}")
print(f"Min: {df['rating'].min()}, Max: {df['rating'].max()}")
print(f"Missing values: {df['rating'].isna().sum()}")
print(f"Average rating: {df['rating'].mean():.2f}")

# Fill missing ratings with the average rating
average_rating = df['rating'].mean()
rows_before = len(df)
df['rating'].fillna(average_rating, inplace=True)
print(f"\nFilled {rows_before - df['rating'].notna().sum()} missing ratings with average ({average_rating:.2f})")
print(f"All ratings now have values: {df['rating'].notna().all()}")

Rating column analysis:
Data type: float64
Min: 2.0, Max: 5.0
Missing values: 1
Average rating: 4.10

Filled 0 missing ratings with average (4.10)
All ratings now have values: True


In [6]:
# Step 5: Split categories into Main Category and Sub Category
# Category format: "Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables"
# Main Category: First part (before first |)
# Sub Category: Last part (after last |)

def extract_main_category(category):
    """Extract main category (first part before |)"""
    if pd.isna(category):
        return np.nan
    parts = str(category).split('|')
    return parts[0].strip()

def extract_sub_category(category):
    """Extract sub category (last part after |)"""
    if pd.isna(category):
        return np.nan
    parts = str(category).split('|')
    return parts[-1].strip()

df['Main_Category'] = df['category'].apply(extract_main_category)
df['Sub_Category'] = df['category'].apply(extract_sub_category)

print("Categories split successfully:")
print(f"Main Categories: {df['Main_Category'].nunique()} unique values")
print(f"Sub Categories: {df['Sub_Category'].nunique()} unique values")
print("\nMain Category distribution:")
print(df['Main_Category'].value_counts())
print("\nSample Sub Categories:")
print(df['Sub_Category'].value_counts().head(10))

Categories split successfully:
Main Categories: 9 unique values
Sub Categories: 207 unique values

Main Category distribution:
Main_Category
Electronics              526
Computers&Accessories    453
Home&Kitchen             448
OfficeProducts            31
MusicalInstruments         2
HomeImprovement            2
Toys&Games                 1
Car&Motorbike              1
Health&PersonalCare        1
Name: count, dtype: int64

Sample Sub Categories:
Sub_Category
USBCables           233
SmartWatches         76
Smartphones          68
SmartTelevisions     63
In-Ear               52
RemoteControls       49
MixerGrinders        27
Mice                 24
HDMICables           24
DryIrons             24
Name: count, dtype: int64


In [7]:
# Step 6: Add calculated columns
# Savings = actual_price - discounted_price
# Discount_Amount = actual_price - discounted_price

df['Savings'] = df['actual_price'] - df['discounted_price']

print("Calculated columns added:")
print(f"Savings column created")
print(f"Min savings: ₹{df['Savings'].min():.2f}")
print(f"Max savings: ₹{df['Savings'].max():.2f}")
print(f"Average savings: ₹{df['Savings'].mean():.2f}")

# Display sample of calculated column
print("\nSample rows with calculated Savings:")
print(df[['product_name', 'actual_price', 'discounted_price', 'Savings']].head(10))

Calculated columns added:
Savings column created
Min savings: ₹0.00
Max savings: ₹61910.00
Average savings: ₹2319.68

Sample rows with calculated Savings:
                                        product_name  actual_price  \
0  Wayona Nylon Braided USB to Lightning Fast Cha...        1099.0   
1  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...         349.0   
2  Sounce Fast Phone Charging Cable & Data Sync U...        1899.0   
3  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...         699.0   
4  Portronics Konnect L 1.2M Fast Charging 3A 8 P...         399.0   
5  pTron Solero TB301 3A Type-C Data and Fast Cha...        1000.0   
6  boAt Micro USB 55 Tangle-free, Sturdy Micro US...         499.0   
7             MI Usb Type-C Cable Smartphone (Black)         299.0   
8  TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...         999.0   
9  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...         299.0   

   discounted_price  Savings  
0            399.00   700.00  
1           

In [8]:
# Step 7: Handle text fields
# review_content and review_title are long text fields
# Truncate for better usability, but keep original for word clouds

# Truncate review_title to first 50 characters
df['review_title_truncated'] = df['review_title'].fillna('').str[:50]

# For review_content, show first 100 characters for preview
df['review_content_truncated'] = df['review_content'].fillna('').str[:100]

# Calculate text lengths for analysis
df['review_title_length'] = df['review_title'].fillna('').str.len()
df['review_content_length'] = df['review_content'].fillna('').str.len()

print("Text field analysis:")
print(f"\nReview Title:")
print(f"  Average length: {df['review_title_length'].mean():.1f} characters")
print(f"  Max length: {df['review_title_length'].max()} characters")
print(f"  Missing values: {df['review_title'].isna().sum()}")

print(f"\nReview Content:")
print(f"  Average length: {df['review_content_length'].mean():.1f} characters")
print(f"  Max length: {df['review_content_length'].max()} characters")
print(f"  Missing values: {df['review_content'].isna().sum()}")

print("\nSample review_title_truncated:")
print(df['review_title_truncated'].head())

Text field analysis:

Review Title:
  Average length: 181.9 characters
  Max length: 407 characters
  Missing values: 0

Review Content:
  Average length: 1394.1 characters
  Max length: 18547 characters
  Missing values: 0

Sample review_title_truncated:
0    Satisfied,Charging is really fast,Value for mo...
1    A Good Braided Cable for Your Type C Device,Go...
2    Good speed for earlier versions,Good Product,W...
3    Good product,Good one,Nice,Really nice product...
4    As good as original,Decent,Good one for second...
Name: review_title_truncated, dtype: object


In [9]:
# Step 8: Create final cleaned dataset
# Select relevant columns for the cleaned dataset

# Columns to keep in cleaned dataset
columns_to_keep = [
    'product_id', 'product_name', 'category', 'Main_Category', 'Sub_Category',
    'discounted_price', 'actual_price', 'discount_percentage', 'Savings',
    'rating', 'rating_count', 'about_product',
    'user_id', 'user_name', 'review_id', 
    'review_title', 'review_title_length', 'review_title_truncated',
    'review_content', 'review_content_length', 'review_content_truncated',
    'img_link', 'product_link'
]

df_cleaned = df[columns_to_keep].copy()

print("Cleaned dataset created:")
print(f"Shape: {df_cleaned.shape}")
print(f"\nColumn names:")
print(df_cleaned.columns.tolist())
print(f"\nData types:")
print(df_cleaned.dtypes)
print(f"\nMissing values in cleaned dataset:")
print(df_cleaned.isnull().sum())
print(f"\nFirst few rows:")
print(df_cleaned.head())

Cleaned dataset created:
Shape: (1465, 23)

Column names:
['product_id', 'product_name', 'category', 'Main_Category', 'Sub_Category', 'discounted_price', 'actual_price', 'discount_percentage', 'Savings', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_title_length', 'review_title_truncated', 'review_content', 'review_content_length', 'review_content_truncated', 'img_link', 'product_link']

Data types:
product_id                   object
product_name                 object
category                     object
Main_Category                object
Sub_Category                 object
discounted_price            float64
actual_price                float64
discount_percentage         float64
Savings                     float64
rating                      float64
rating_count                float64
about_product                object
user_id                      object
user_name                    object
review_id                    object

In [10]:
# Step 9: Save cleaned dataset as CSV with UTF-8 encoding
output_file = 'amazon_cleaned.csv'

df_cleaned.to_csv(output_file, index=False, encoding='utf-8')

print(f"✓ Cleaned dataset saved successfully!")
print(f"  File: {output_file}")
print(f"  Rows: {df_cleaned.shape[0]}")
print(f"  Columns: {df_cleaned.shape[1]}")
print(f"  Encoding: UTF-8")

# Verify the file was created
import os
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"  File size: {file_size:,} bytes")
    print(f"\n✓ File created successfully!")

✓ Cleaned dataset saved successfully!
  File: amazon_cleaned.csv
  Rows: 1465
  Columns: 23
  Encoding: UTF-8
  File size: 5,033,301 bytes

✓ File created successfully!


In [11]:
# Step 10: Data quality summary report
print("=" * 70)
print("DATA CLEANING SUMMARY REPORT")
print("=" * 70)

print("\n1. PRICE COLUMNS:")
print(f"   ✓ Removed currency symbol (₹) and commas")
print(f"   ✓ Converted to numeric (float)")
print(f"   Discounted Price - Range: ₹{df_cleaned['discounted_price'].min():.0f} to ₹{df_cleaned['discounted_price'].max():.0f}")
print(f"   Actual Price - Range: ₹{df_cleaned['actual_price'].min():.0f} to ₹{df_cleaned['actual_price'].max():.0f}")

print("\n2. DISCOUNT PERCENTAGE:")
print(f"   ✓ Removed % symbol and commas")
print(f"   ✓ Converted to numeric (float)")
print(f"   Range: {df_cleaned['discount_percentage'].min():.0f}% to {df_cleaned['discount_percentage'].max():.0f}%")

print("\n3. RATING COUNT:")
print(f"   ✓ Removed commas")
print(f"   ✓ Converted to numeric (integer)")
print(f"   Range: {int(df_cleaned['rating_count'].min()):,} to {int(df_cleaned['rating_count'].max()):,} ratings")

print("\n4. RATING:")
print(f"   ✓ Ensured numeric format")
print(f"   ✓ Filled missing values with average ({average_rating:.2f})")
print(f"   Range: {df_cleaned['rating'].min():.1f} to {df_cleaned['rating'].max():.1f} stars")
print(f"   No missing values: {df_cleaned['rating'].notna().all()}")

print("\n5. CATEGORIES:")
print(f"   ✓ Split into Main Category (first part) and Sub Category (last part)")
print(f"   Main Categories: {df_cleaned['Main_Category'].nunique()} unique")
print(f"   Sub Categories: {df_cleaned['Sub_Category'].nunique()} unique")

print("\n6. CALCULATED COLUMNS:")
print(f"   ✓ Savings = actual_price - discounted_price")
print(f"   Average Savings: ₹{df_cleaned['Savings'].mean():.2f}")

print("\n7. TEXT FIELDS:")
print(f"   ✓ Created truncated versions for display (50 chars for title, 100 for content)")
print(f"   ✓ Created length columns for analysis")
print(f"   Review titles: {(df_cleaned['review_title'].notna().sum() / len(df_cleaned) * 100):.1f}% populated")
print(f"   Review content: {(df_cleaned['review_content'].notna().sum() / len(df_cleaned) * 100):.1f}% populated")

print("\n8. OUTPUT FILE:")
print(f"   ✓ Saved as: amazon_cleaned.csv")
print(f"   ✓ Encoding: UTF-8")
print(f"   ✓ Rows: {len(df_cleaned):,}")
print(f"   ✓ Columns: {len(df_cleaned.columns)}")

print("\n" + "=" * 70)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("=" * 70)

DATA CLEANING SUMMARY REPORT

1. PRICE COLUMNS:
   ✓ Removed currency symbol (₹) and commas
   ✓ Converted to numeric (float)
   Discounted Price - Range: ₹39 to ₹77990
   Actual Price - Range: ₹39 to ₹139900

2. DISCOUNT PERCENTAGE:
   ✓ Removed % symbol and commas
   ✓ Converted to numeric (float)
   Range: 0% to 94%

3. RATING COUNT:
   ✓ Removed commas
   ✓ Converted to numeric (integer)
   Range: 2 to 426,973 ratings

4. RATING:
   ✓ Ensured numeric format
   ✓ Filled missing values with average (4.10)
   Range: 2.0 to 5.0 stars
   No missing values: True

5. CATEGORIES:
   ✓ Split into Main Category (first part) and Sub Category (last part)
   Main Categories: 9 unique
   Sub Categories: 207 unique

6. CALCULATED COLUMNS:
   ✓ Savings = actual_price - discounted_price
   Average Savings: ₹2319.68

7. TEXT FIELDS:
   ✓ Created truncated versions for display (50 chars for title, 100 for content)
   ✓ Created length columns for analysis
   Review titles: 100.0% populated
   Review c